# AG2: Structured outputs

- Structured outputs<br>
  - https://docs.ag2.ai/docs/user-guide/basic-concepts/llm-configuration/structured-outputs
  - https://docs.ag2.ai/docs/use-cases/notebooks/notebooks/agentchat_structured_outputs

## SETUP

In [1]:
import os
from dotenv import load_dotenv

# Load environment variables (for API key)
load_dotenv()

# Set up OpenAI API key
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("Please set the OPENAI_API_KEY environment variable or add it to a .env file")

# Define the model to use
MODEL_GPT = "gpt-4o-mini"

## Installing AG2
We recommended using a virtual environment for your project to keep your packages contained.<br>
See **venv**<br>
https://docs.python.org/3/library/venv.html

Install AG2 with your preferred model provider(s), for example:
```
pip install ag2[openai]
pip install ag2[gemini]
pip install ag2[anthropic,cohere,mistral]
```

If you have been using autogen or ag2, all you need to do is upgrade it using:
```
pip install -U autogen[openai]
pip install -U ag2[openai]
```
as **ag2**, **autogen**, and **ag2** are aliases for the same PyPI package.

## LLM Configuration

```
import os

llm_config = {
  "config_list": [
    {
      "api_type": "openai",
      "model": "gpt-4o-mini",
      "api_key": os.environ["OPENAI_API_KEY"]
    }
  ],
}

my_agent = ConversableAgent(
    name="helpful_agent",
    llm_config=llm_config,
    system_message="You are a poetic AI assistant",
)
```

## Create lesson plan (using structured outputs)

In [2]:
import json
from pydantic import BaseModel
from autogen import ConversableAgent

In [3]:
# 1. Define our lesson plan structure, a lesson with a number of objectives
class LearningObjective(BaseModel):
    title: str
    description: str


class LessonPlan(BaseModel):
    title: str
    learning_objectives: list[LearningObjective]
    script: str


# 2. Add our lesson plan structure to the LLM configuration
llm_config = {
    "api_type": "openai",
    "model": "gpt-4o-mini",
    "response_format": LessonPlan,
}

# 3. The agent's system message doesn't need any formatting instructions
system_message = """You are a classroom lesson agent.
Given a topic, write a lesson plan for a fourth grade class.
"""

my_agent = ConversableAgent(
    name="lesson_agent",
    llm_config=llm_config,
    system_message=system_message
    )

In [4]:
# 4. Chat directly with our agent
chat_result = my_agent.run("In one sentence, what's the big deal about AI?")

user (to lesson_agent):

In one sentence, what's the big deal about AI?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
lesson_agent (to user):

{"title":"Understanding Artificial Intelligence","learning_objectives":[{"title":"Identify AI in Daily Life","description":"Students will be able to recognize examples of artificial intelligence they encounter in their daily lives."},{"title":"Explain How AI Works","description":"Students will understand the basic principles of how artificial intelligence operates."},{"title":"Discuss the Impact of AI","description":"Students will reflect on the positive and negative impacts of artificial intelligence on society."}],"script":"Today, we will explore the exciting world of artificial intelligence, or AI. Many of you might have already encountered AI in some form, whether it's through smart assistants like Siri or Alexa, recommendation systems for your favorite shows, or even when you

Replying as user. Provide feedback to lesson_agent. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  exit


In [5]:
# 5. Get and print our lesson plan
lesson_plan_json = json.loads(chat_result.chat_history[-1]["content"])
print(json.dumps(lesson_plan_json, indent=2))

{
  "title": "Understanding Artificial Intelligence",
  "learning_objectives": [
    {
      "title": "Identify AI in Daily Life",
      "description": "Students will be able to recognize examples of artificial intelligence they encounter in their daily lives."
    },
    {
      "title": "Explain How AI Works",
      "description": "Students will understand the basic principles of how artificial intelligence operates."
    },
    {
      "title": "Discuss the Impact of AI",
      "description": "Students will reflect on the positive and negative impacts of artificial intelligence on society."
    }
  ],
  "script": "Today, we will explore the exciting world of artificial intelligence, or AI. Many of you might have already encountered AI in some form, whether it's through smart assistants like Siri or Alexa, recommendation systems for your favorite shows, or even when you're playing video games. We will begin by discussing what AI is and how it's become a part of our everyday lives. Th

## Math reasoning (using structured outputs)

In [6]:
import autogen
from pydantic import BaseModel

In [7]:
# Set your API Endpoint
# config_list = autogen.config_list_from_json(
#     "OAI_CONFIG_LIST",
#     filter_dict={
#         "model": ["gpt-4o", "gpt-4o-mini"],
#     },
# )

In [8]:
# Define the reasoning model
class Step(BaseModel):
    explanation: str
    output: str

class MathReasoning(BaseModel):
    steps: list[Step]
    final_answer: str

In [9]:
# Define chat actors
llm_config = {
    # "config_list": config_list,
    "api_type": "openai",
    "model": "gpt-4o-mini",
    "cache_seed": 42,
    "response_format": MathReasoning,
}

user_proxy = autogen.UserProxyAgent(
    name="User_proxy",
    system_message="A human admin.",
    human_input_mode="NEVER",
    code_execution_config={
        "work_dir": "coding_work",
        "use_docker": False,  # Set to True if you want to use Docker
    }
)

assistant = autogen.AssistantAgent(
    name="Math_solver",
    llm_config=llm_config,  # Response Format is in the configuration
)

In [10]:
# Start the chat
user_proxy.initiate_chat(assistant, message="how can I solve 8x + 7 = -23", max_turns=1, summary_method="last_msg")

User_proxy (to Math_solver):

how can I solve 8x + 7 = -23

--------------------------------------------------------------------------------
Math_solver (to User_proxy):

{"steps":[{"explanation":"First, we need to isolate the term with 'x'. We can do this by subtracting 7 from both sides of the equation.","output":"8x + 7 - 7 = -23 - 7"},{"explanation":"This simplifies to 8x = -30. Now, we need to solve for 'x' by dividing both sides by 8.","output":"8x / 8 = -30 / 8"},{"explanation":"This gives us x = -30/8. We can simplify this further by dividing both the numerator and the denominator by 2.","output":"x = -15/4 or x = -3.75."}],"final_answer":"x = -3.75"}

--------------------------------------------------------------------------------


ChatResult(chat_id=None, chat_history=[{'content': 'how can I solve 8x + 7 = -23', 'role': 'assistant', 'name': 'User_proxy'}, {'content': '{"steps":[{"explanation":"First, we need to isolate the term with \'x\'. We can do this by subtracting 7 from both sides of the equation.","output":"8x + 7 - 7 = -23 - 7"},{"explanation":"This simplifies to 8x = -30. Now, we need to solve for \'x\' by dividing both sides by 8.","output":"8x / 8 = -30 / 8"},{"explanation":"This gives us x = -30/8. We can simplify this further by dividing both the numerator and the denominator by 2.","output":"x = -15/4 or x = -3.75."}],"final_answer":"x = -3.75"}', 'role': 'user', 'name': 'Math_solver'}], summary='{"steps":[{"explanation":"First, we need to isolate the term with \'x\'. We can do this by subtracting 7 from both sides of the equation.","output":"8x + 7 - 7 = -23 - 7"},{"explanation":"This simplifies to 8x = -30. Now, we need to solve for \'x\' by dividing both sides by 8.","output":"8x / 8 = -30 / 8"}

## Math reasoning (using structured outputs, formatting response)

In [11]:
import autogen
from pydantic import BaseModel

In [12]:
# Define the reasoning model
class Step(BaseModel):
    explanation: str
    output: str

class MathReasoning(BaseModel):
    steps: list[Step]
    final_answer: str

    def format(self) -> str:
        steps_output = "\n".join(
            f"Step {i + 1}: {step.explanation}\n  Output: {step.output}" for i, step in enumerate(self.steps)
        )
        return f"{steps_output}\n\nFinal Answer: {self.final_answer}"

In [13]:
# Define chat actors and start the chat
# for config in config_list:
#     config["response_format"] = MathReasoning
llm_config = {
    # "config_list": config_list,
    "api_type": "openai",
    "model": "gpt-4o-mini",
    "cache_seed": 42,
    "response_format": MathReasoning,
}

user_proxy = autogen.UserProxyAgent(
    name="User_proxy",
    system_message="A human admin.",
    human_input_mode="NEVER",
    code_execution_config={
        "work_dir": "coding_work",
        "use_docker": False,  # Set to True if you want to use Docker
    }
)

assistant = autogen.AssistantAgent(
    name="Math_solver",
    llm_config=llm_config,
)

print(
    user_proxy.initiate_chat(
        assistant, message="how can I solve 8x + 7 = -23", max_turns=1, summary_method="last_msg"
    ).summary
)

User_proxy (to Math_solver):

how can I solve 8x + 7 = -23

--------------------------------------------------------------------------------
Math_solver (to User_proxy):

Step 1: First, we need to isolate the term with 'x'. We can do this by subtracting 7 from both sides of the equation.
  Output: 8x + 7 - 7 = -23 - 7
Step 2: This simplifies to 8x = -30. Now, we need to solve for 'x' by dividing both sides by 8.
  Output: 8x / 8 = -30 / 8
Step 3: This gives us x = -30/8. We can simplify this further by dividing both the numerator and the denominator by 2.
  Output: x = -15/4 or x = -3.75.

Final Answer: x = -3.75

--------------------------------------------------------------------------------
Step 1: First, we need to isolate the term with 'x'. We can do this by subtracting 7 from both sides of the equation.
  Output: 8x + 7 - 7 = -23 - 7
Step 2: This simplifies to 8x = -30. Now, we need to solve for 'x' by dividing both sides by 8.
  Output: 8x / 8 = -30 / 8
Step 3: This gives us x =

In [14]:
@user_proxy.register_for_execution()
@assistant.register_for_llm(description="You can use this function call to solve addition")
def add(x: int, y: int) -> int:
    return x + y

user_proxy.initiate_chat(
    assistant, message="solve 3 + 4 by calling appropriate function", max_turns=2, summary_method="last_msg"
)

User_proxy (to Math_solver):

solve 3 + 4 by calling appropriate function

--------------------------------------------------------------------------------
Math_solver (to User_proxy):

***** Suggested tool call (call_8RfUeVkbFY8JCtBr7Smk6IEG): add *****
Arguments: 
{"x":3,"y":4}
********************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION add...
Call ID: call_8RfUeVkbFY8JCtBr7Smk6IEG
Input arguments: {'x': 3, 'y': 4}
User_proxy (to Math_solver):

***** Response from calling tool (call_8RfUeVkbFY8JCtBr7Smk6IEG) *****
7
**********************************************************************

--------------------------------------------------------------------------------
Math_solver (to User_proxy):

Step 1: I used the add function to compute the sum of 3 and 4.
  Output: Result from the add function: 7

Final Answer: The sum of 3 + 4 is 7.

-----------------------------

ChatResult(chat_id=None, chat_history=[{'content': 'solve 3 + 4 by calling appropriate function', 'role': 'assistant', 'name': 'User_proxy'}, {'tool_calls': [{'id': 'call_8RfUeVkbFY8JCtBr7Smk6IEG', 'function': {'arguments': '{"x":3,"y":4}', 'name': 'add'}, 'type': 'function'}], 'content': None, 'role': 'assistant'}, {'content': '7', 'tool_responses': [{'tool_call_id': 'call_8RfUeVkbFY8JCtBr7Smk6IEG', 'role': 'tool', 'content': '7'}], 'role': 'tool', 'name': 'User_proxy'}, {'content': 'Step 1: I used the add function to compute the sum of 3 and 4.\n  Output: Result from the add function: 7\n\nFinal Answer: The sum of 3 + 4 is 7.', 'role': 'user', 'name': 'Math_solver'}], summary='Step 1: I used the add function to compute the sum of 3 and 4.\n  Output: Result from the add function: 7\n\nFinal Answer: The sum of 3 + 4 is 7.', cost={'usage_including_cached_inference': {'total_cost': 0.00023505, 'gpt-4o-mini-2024-07-18': {'cost': 0.00023505, 'prompt_tokens': 1263, 'completion_tokens': 76, 